# Agent: People (Phase 4)

This notebook subscribes to control commands and publishes person state/position updates over MQTT.


In [ ]:
# Cell purpose: import dependencies, load config, and initialize MQTT + simulation constants.
from __future__ import annotations

from datetime import datetime, timezone
import json
import math
import random
import time

from simulated_city.config import load_config
from simulated_city.models import Person, PersonState
from simulated_city.movement import apply_boundary_bounce, random_walk_step
from simulated_city.mqtt import MqttConnector, MqttPublisher
from simulated_city.routing import step_toward_target

config = load_config()
simulation_cfg = config.simulation
if simulation_cfg is None:
    raise ValueError("config.yaml must include a 'simulation' section for this notebook")

rng = random.Random(simulation_cfg.seed if simulation_cfg.seed is not None else 42)

PEOPLE_COUNT = simulation_cfg.people_count
STEP_DISTANCE = simulation_cfg.movement.step_distance_m
MAX_TURN_DEG = simulation_cfg.movement.max_turn_deg
TICK_SECONDS = simulation_cfg.movement.tick_s
TOTAL_TICKS = simulation_cfg.movement.total_ticks

MIN_X, MAX_X = simulation_cfg.map.min_x, simulation_cfg.map.max_x
MIN_Y, MAX_Y = simulation_cfg.map.min_y, simulation_cfg.map.max_y

COMMAND_SUBSCRIPTION = "city/people/+/command"
STATE_TOPIC_TEMPLATE = "city/people/{person_id}/state"
POSITION_TOPIC_TEMPLATE = "city/people/{person_id}/position"
AGGREGATE_TOPIC = "city/people/aggregate"

print(f"People agent broker: {config.mqtt.host}:{config.mqtt.port} (tls={config.mqtt.tls})")


In [ ]:
# Cell purpose: initialize people and command-state handlers.
names = list(simulation_cfg.names)
colors = list(simulation_cfg.colors)

people: list[Person] = []
people_by_id: dict[str, Person] = {}
for index in range(PEOPLE_COUNT):
    person = Person(
        person_id=f"person_{index + 1}",
        name=names[index % len(names)],
        color=colors[index % len(colors)],
        x=rng.uniform(MIN_X, MAX_X),
        y=rng.uniform(MIN_Y, MAX_Y),
        heading_deg=rng.uniform(0.0, 360.0),
        state=PersonState.RANDOM_WALK,
    )
    people.append(person)
    people_by_id[person.person_id] = person

target_by_person: dict[str, dict[str, str | float] | None] = {p.person_id: None for p in people}

connector = MqttConnector(config.mqtt, client_id_suffix="people")
connector.connect()
if not connector.wait_for_connection(timeout=10.0):
    raise TimeoutError("People agent could not connect to MQTT within timeout")
publisher = MqttPublisher(connector)

def _utc_timestamp_iso() -> str:
    return datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")

def _command_mode_to_state(mode: str) -> PersonState:
    normalized = mode.strip().lower()
    if normalized == "move_to_shop":
        return PersonState.SEEKING_SHELTER
    return PersonState.RANDOM_WALK

def on_message(client, userdata, msg):
    if not (msg.topic.startswith("city/people/") and msg.topic.endswith("/command")):
        return

    try:
        payload = json.loads(msg.payload.decode("utf-8"))
    except json.JSONDecodeError:
        return

    person_id = str(payload.get("person_id") or msg.topic.split("/")[2])
    person = people_by_id.get(person_id)
    if person is None:
        return

    mode = str(payload.get("mode", "random_walk"))
    person.state = _command_mode_to_state(mode)
    raw_target = payload.get("target_shop")
    target_by_person[person_id] = raw_target if isinstance(raw_target, dict) else None

connector.client.on_message = on_message
connector.client.subscribe(COMMAND_SUBSCRIPTION, qos=1)

print(f"Subscribed to command topic: {COMMAND_SUBSCRIPTION}")
print(f"Initialized {len(people)} people and waiting for control commands.")


In [ ]:
# Cell purpose: run the movement loop, execute commands, and publish state/position updates.
try:
    for tick in range(1, TOTAL_TICKS + 1):
        aggregate_people: list[dict[str, str | float | int]] = []
        print(f"\nTick {tick}")

        for person in people:
            if person.state == PersonState.RANDOM_WALK:
                x1, y1, heading1 = random_walk_step(
                    person.x,
                    person.y,
                    person.heading_deg,
                    step_distance=STEP_DISTANCE,
                    max_turn_deg=MAX_TURN_DEG,
                    rng=rng,
                )
                x2, y2, heading2 = apply_boundary_bounce(
                    x1, y1, heading1, MIN_X, MAX_X, MIN_Y, MAX_Y
                )
                person.x = x2
                person.y = y2
                person.heading_deg = heading2
            else:
                target = target_by_person.get(person.person_id)
                if isinstance(target, dict):
                    target_x = float(target.get("x", person.x))
                    target_y = float(target.get("y", person.y))
                    prev_x = person.x
                    prev_y = person.y
                    next_x, next_y, _arrived = step_toward_target(
                        x=person.x,
                        y=person.y,
                        target_x=target_x,
                        target_y=target_y,
                        step_distance=STEP_DISTANCE,
                    )
                    person.heading_deg = (math.degrees(math.atan2(next_y - prev_y, next_x - prev_x)) + 360.0) % 360.0
                    person.x = next_x
                    person.y = next_y

            position_payload = {
                "source": "agent_people",
                "person_id": person.person_id,
                "x": round(person.x, 3),
                "y": round(person.y, 3),
                "heading_deg": round(person.heading_deg % 360.0, 2),
                "tick": tick,
                "timestamp": _utc_timestamp_iso(),
            }
            state_payload = {
                "source": "agent_people",
                "person_id": person.person_id,
                "state": person.state.value,
                "tick": tick,
                "timestamp": _utc_timestamp_iso(),
            }

            publisher.publish_json(
                POSITION_TOPIC_TEMPLATE.format(person_id=person.person_id),
                json.dumps(position_payload),
                qos=0,
                retain=False,
            )
            publisher.publish_json(
                STATE_TOPIC_TEMPLATE.format(person_id=person.person_id),
                json.dumps(state_payload),
                qos=0,
                retain=False,
            )

            aggregate_people.append(
                {
                    "person_id": person.person_id,
                    "name": person.name,
                    "color": person.color,
                    "x": round(person.x, 3),
                    "y": round(person.y, 3),
                    "state": person.state.value,
                }
            )

            log = person.to_log_dict()
            print(
                f"- {log['person_id']} ({log['name']}, {log['color']}): x={log['x']}, y={log['y']}, heading={log['heading_deg']}, state={log['state']}"
            )

        aggregate_payload = {
            "source": "agent_people",
            "tick": tick,
            "people": aggregate_people,
            "timestamp": _utc_timestamp_iso(),
        }
        publisher.publish_json(AGGREGATE_TOPIC, json.dumps(aggregate_payload), qos=0, retain=False)

        time.sleep(TICK_SECONDS)
finally:
    connector.disconnect()
    print("People agent disconnected from MQTT.")
